## Multiple object tracking with YOLOv3-based object detection

In [2]:
import numpy as np
import cv2 as cv
from motrackers.detectors import YOLOv3
from motrackers import CentroidTracker, CentroidKF_Tracker, SORT, IOUTracker
from motrackers.utils import draw_tracks
import ipywidgets as widgets
import csv

In [14]:
# VIDEO_FILE = "./../video_data/cars.mp4"
WEIGHTS_PATH = './../pretrained_models/yolo_weights/yolov3.weights'
CONFIG_FILE_PATH = './../pretrained_models/yolo_weights/yolov3.cfg'
LABELS_PATH = "./../pretrained_models/yolo_weights/coco_names.json"

CONFIDENCE_THRESHOLD = 0.5
NMS_THRESHOLD = 0.2
DRAW_BOUNDING_BOXES = True
USE_GPU = False

chosen_tracker = widgets.Select(
    options=["CentroidTracker", "CentroidKF_Tracker", "SORT", "IOUTracker"],
    value='IOUTracker',
    rows=5,
    description='MOTracker:',
    disabled=False
)
chosen_tracker

if chosen_tracker.value == 'CentroidTracker':
    tracker = CentroidTracker(max_lost=0, tracker_output_format='mot_challenge')
elif chosen_tracker.value == 'CentroidKF_Tracker':
    tracker = CentroidKF_Tracker(max_lost=0, tracker_output_format='mot_challenge')
elif chosen_tracker.value == 'SORT':
    tracker = SORT(max_lost=3, tracker_output_format='mot_challenge', iou_threshold=0.3)
elif chosen_tracker.value == 'IOUTracker':
    tracker = IOUTracker(max_lost=2, iou_threshold=0.5, min_detection_confidence=0.4, max_detection_confidence=0.7,
                         tracker_output_format='mot_challenge')
else:
    print("Please choose one tracker from the above list.")

model = YOLOv3(
    weights_path=WEIGHTS_PATH,
    configfile_path=CONFIG_FILE_PATH,
    labels_path=LABELS_PATH,
    confidence_threshold=CONFIDENCE_THRESHOLD,
    nms_threshold=NMS_THRESHOLD,
    draw_bboxes=DRAW_BOUNDING_BOXES,
    use_gpu=USE_GPU
)

def main(video_path, model, tracker):
    
    cap = cv.VideoCapture(video_path)
    length = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

    out = cv.VideoWriter('result.mp4', -1, 20.0, (700,500))
    with open ('results.csv', 'w') as f:
        thewriter = csv.writer(f)
        thewriter.writerow(['frame_number', 'objects', "tracks"])
        for frame_number in  range(1, length+1):
            ok, image = cap.read()

            if not ok:
                print("Cannot read the video feed.")
                break
            image = cv.resize(image, (700, 500))
            bboxes, confidences, class_ids = model.detect(image)
            
            tracks = tracker.update(bboxes, confidences, class_ids)
            updated_image, all_label = model.draw_bboxes(image.copy(), bboxes, confidences, class_ids)#comes from detector/do nothing
            
            print(updated_image.dtype)
            print(updated_image.shape)
            updated_image, tracks_all = draw_tracks(updated_image, tracks)
            
            out.write(updated_image)
            cv.imshow("liora3", updated_image)
            thewriter.writerow([frame_number, all_label, tracks_all])
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        


    cap.release()
    out.release()
    cv.destroyAllWindows()

main(0, model, tracker)

In [5]:
# VIDEO_FILE = "./../video_data/cars.mp4"
WEIGHTS_PATH = './../pretrained_models/yolo_weights/yolov3.weights'
CONFIG_FILE_PATH = './../pretrained_models/yolo_weights/yolov3.cfg'
LABELS_PATH = "./../pretrained_models/yolo_weights/coco_names.json"

CONFIDENCE_THRESHOLD = 0.5
NMS_THRESHOLD = 0.2
DRAW_BOUNDING_BOXES = True
USE_GPU = False

chosen_tracker = widgets.Select(
    options=["CentroidTracker", "CentroidKF_Tracker", "SORT", "IOUTracker"],
    value='IOUTracker',
    rows=5,
    description='MOTracker:',
    disabled=False
)
chosen_tracker

if chosen_tracker.value == 'CentroidTracker':
    tracker = CentroidTracker(max_lost=0, tracker_output_format='mot_challenge')
elif chosen_tracker.value == 'CentroidKF_Tracker':
    tracker = CentroidKF_Tracker(max_lost=0, tracker_output_format='mot_challenge')
elif chosen_tracker.value == 'SORT':
    tracker = SORT(max_lost=3, tracker_output_format='mot_challenge', iou_threshold=0.3)
elif chosen_tracker.value == 'IOUTracker':
    tracker = IOUTracker(max_lost=2, iou_threshold=0.5, min_detection_confidence=0.4, max_detection_confidence=0.7,
                         tracker_output_format='mot_challenge')
else:
    print("Please choose one tracker from the above list.")

model = YOLOv3(
    weights_path=WEIGHTS_PATH,
    configfile_path=CONFIG_FILE_PATH,
    labels_path=LABELS_PATH,
    confidence_threshold=CONFIDENCE_THRESHOLD,
    nms_threshold=NMS_THRESHOLD,
    draw_bboxes=DRAW_BOUNDING_BOXES,
    use_gpu=USE_GPU
)

def main(video_path, model, tracker):
    
    cap = cv.VideoCapture(video_path)
    length = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

    out = cv.VideoWriter('result.mp4', -1, 20.0, (700,500))
    with open ('results.csv', 'w') as f:
        thewriter = csv.writer(f)
        thewriter.writerow(['frame_number', 'objects', "tracks"])
        count = 1
        while True:
    #     for frame_number in  range(1, length+1):
            ok, image = cap.read()

            if not ok:
                print("Cannot read the video feed.")
                break
            image = cv.resize(image, (700, 500))
            bboxes, confidences, class_ids = model.detect(image)

            tracks = tracker.update(bboxes, confidences, class_ids)
            updated_image, all_label = model.draw_bboxes(image.copy(), bboxes, confidences, class_ids)#comes from detector/do nothing

            print(updated_image.dtype)
            print(updated_image.shape)
            updated_image, tracks_all = draw_tracks(updated_image, tracks)

            out.write(updated_image)
            cv.imshow("liora3", updated_image)
            thewriter.writerow([count, all_label, tracks_all])
            count+1
            if cv.waitKey(1) & 0xFF == ord('q'):
                break



        cap.release()
        out.release()
        cv.destroyAllWindows()

main("./../video_data/cars.mp4" , model, tracker)

uint8
(500, 700, 3)
uint8
(500, 700, 3)
uint8
(500, 700, 3)
uint8
(500, 700, 3)


In [22]:
 cv.destroyAllWindows()

In [17]:
 cap.release()

NameError: name 'cap' is not defined

In [18]:
    out.release()


NameError: name 'out' is not defined